In [1]:
import pandas as pd  #where from the cluster is that information?

In [2]:
maig = pd.read_csv("/home/roger/send_data/cluster/2025-05.csv", sep="|")

In [3]:
df = maig.copy()

In [4]:
df

,JobID,User,Start,End,Elapsed,State,AllocCPUS,NodeList
0,21527,10033944@uvic.local,2025-04-29T10:31:26,2025-05-21T11:08:58,22-00:37:32,FAILED,1,vcompute11
1,21528,nsekhar,2025-04-30T11:44:46,2025-05-02T00:55:36,1-13:10:50,COMPLETED,1,vcompute11
2,21529,nsekhar,2025-04-30T11:44:46,2025-05-02T00:56:51,1-13:12:05,COMPLETED,1,vcompute11
3,21530,nsekhar,2025-04-30T11:44:46,2025-05-02T01:28:17,1-13:43:31,COMPLETED,1,vcompute11
4,21531,nsekhar,2025-04-30T11:44:46,2025-05-02T01:30:49,1-13:46:03,COMPLETED,1,vcompute11
...,...,...,...,...,...,...,...,...
3730,25434,rcasals,2025-05-30T11:33:55,2025-05-30T11:42:03,00:08:08,FAILED,14,vcompute12
3731,25435,rcasals,2025-05-30T11:48:20,Unknown,13:11:42,RUNNING,14,vcompute12
3732,25436,rcasals,2025-05-30T12:31:50,2025-05-30T17:39:27,05:07:37,COMPLETED,40,compute02
3733,25437,rcasals,2025-05-30T13:51:49,Unknown,11:08:13,RUNNING,42,vcompute11


In [5]:
import pandas as pd
import re
import numpy as np

# Load your dataframe (assuming it's already called df)

# Step 1: Parse the "Elapsed" column to seconds
def elapsed_to_seconds(elapsed_str):
    if elapsed_str in ['Unknown', '']:
        return 0
    # Match either D-HH:MM:SS or HH:MM:SS
    match = re.match(r'(?:(\d+)-)?(\d+):(\d+):(\d+)', elapsed_str)
    if match:
        days = int(match.group(1)) if match.group(1) else 0
        hours = int(match.group(2))
        minutes = int(match.group(3))
        seconds = int(match.group(4))
        return days * 86400 + hours * 3600 + minutes * 60 + seconds
    return 0

df['ElapsedSeconds'] = df['Elapsed'].apply(elapsed_to_seconds)

# Step 2: Calculate CPU time (in core-seconds)
df['CPUTimeSeconds'] = df['ElapsedSeconds'] * df['AllocCPUS']

# Optional: Convert to hours
df['CPUTimeHours'] = df['CPUTimeSeconds'] / 3600

# Step 3: Group by user
usage_by_user = df.groupby('User')[['CPUTimeHours']].sum().sort_values(by='CPUTimeHours', ascending=False)


total_cluster_hours = 140 * (720/5)  # Total CPU-hours available in the month
cpu_used = df['CPUTimeHours'].sum()
percent_used = (cpu_used / total_cluster_hours) * 100  #cause we have also gpu so idk how can we compute that.



In [6]:
df

,JobID,User,Start,End,Elapsed,State,AllocCPUS,NodeList,ElapsedSeconds,CPUTimeSeconds,CPUTimeHours
0,21527,10033944@uvic.local,2025-04-29T10:31:26,2025-05-21T11:08:58,22-00:37:32,FAILED,1,vcompute11,1903052,1903052,528.625556
1,21528,nsekhar,2025-04-30T11:44:46,2025-05-02T00:55:36,1-13:10:50,COMPLETED,1,vcompute11,133850,133850,37.180556
2,21529,nsekhar,2025-04-30T11:44:46,2025-05-02T00:56:51,1-13:12:05,COMPLETED,1,vcompute11,133925,133925,37.201389
3,21530,nsekhar,2025-04-30T11:44:46,2025-05-02T01:28:17,1-13:43:31,COMPLETED,1,vcompute11,135811,135811,37.725278
4,21531,nsekhar,2025-04-30T11:44:46,2025-05-02T01:30:49,1-13:46:03,COMPLETED,1,vcompute11,135963,135963,37.767500
...,...,...,...,...,...,...,...,...,...,...,...
3730,25434,rcasals,2025-05-30T11:33:55,2025-05-30T11:42:03,00:08:08,FAILED,14,vcompute12,488,6832,1.897778
3731,25435,rcasals,2025-05-30T11:48:20,Unknown,13:11:42,RUNNING,14,vcompute12,47502,665028,184.730000
3732,25436,rcasals,2025-05-30T12:31:50,2025-05-30T17:39:27,05:07:37,COMPLETED,40,compute02,18457,738280,205.077778
3733,25437,rcasals,2025-05-30T13:51:49,Unknown,11:08:13,RUNNING,42,vcompute11,40093,1683906,467.751667


In [7]:
usage_by_user

,CPUTimeHours
User,
10011847@uvic.local,107832.250556
nsekhar,44781.088611
rcasals,18158.285833
10033944@uvic.local,848.126667
gruiz,653.980000
10010075@uvic.local,50.659444
bruce,0.005556


In [8]:
percent_used

854.7837136243386

In [9]:
# 'Start' (string), 'ElapsedSeconds' (int), 'AllocCPUS' (int)

# Step 1: Convert 'Start' to datetime
df['Start_dt'] = pd.to_datetime(df['Start'], errors='coerce')

# Step 2: Calculate CPU time in hours (if not already)
df['CPUTimeHours'] = (df['ElapsedSeconds'] * df['AllocCPUS']) / 3600

# Step 3: Extract week number (ISO year + week for uniqueness)
df['Week'] = df['Start_dt'].dt.strftime('%Y-%U')

# Step 4: Group by week and get maximum CPU time per job in that week
max_cpu_by_week = df.groupby('Week')['CPUTimeHours'].max().reset_index()

# Optional: sort by week
max_cpu_by_week = max_cpu_by_week.sort_values('Week')

In [10]:
max_cpu_by_week

,Week,CPUTimeHours
0,2025-17,528.625556
1,2025-18,1234.833333
2,2025-19,1529.177778
3,2025-20,1573.622222
4,2025-21,1493.622222
